In [ ]:
# Q&A Assistant using RAG and LangChain Agentic AI

# Required libraries:
# pip install langchain openai faiss-cpu python-dotenv

import os
from dotenv import load_dotenv
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.agents import initialize_agent, Tool
from langchain.memory import ConversationBufferMemory

# Load environment variables (e.g., OpenAI API Key)
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

# Step 1: Load and preprocess documents
def load_documents(file_path):
    loader = TextLoader(file_path)
    documents = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    return splitter.split_documents(documents)

# Step 2: Create a FAISS vector store from documents
def create_vector_index(documents):
    embeddings = OpenAIEmbeddings()
    return FAISS.from_documents(documents, embeddings)

# Step 3: Build a retrieval-based QA chain
def build_qa_chain(vector_store):
    retriever = vector_store.as_retriever(search_type="similarity", k=3)
    llm = ChatOpenAI(model_name="gpt-4", temperature=0)
    return RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff")

# Step 4: Define the tool for the agent to use
def create_tool(qa_chain):
    return [
        Tool(
            name="ContextualQA",
            func=qa_chain.run,
            description="Use this to answer questions based on uploaded knowledge base documents."
        )
    ]

# Step 5: Initialize the agent
def initialize_qa_agent(tools):
    llm = ChatOpenAI(model_name="gpt-4")
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    return initialize_agent(tools, llm, agent="chat-conversational-react-description", verbose=True, memory=memory)

# Step 6: Main execution loop for interactive Q&A
def run_qa_assistant():
    print("\n🤖 Q&A Assistant Ready. Ask your question below.")
    print("Type 'exit' to quit.\n")

    # Load and prepare knowledge base
    documents = load_documents("data/knowledge_base.txt")
    vector_store = create_vector_index(documents)
    qa_chain = build_qa_chain(vector_store)
    tools = create_tool(qa_chain)
    agent = initialize_qa_agent(tools)

    while True:
        question = input("You: ")
        if question.lower() in ["exit", "quit"]:
            print("Assistant: Goodbye!")
            break
        response = agent.run(question)
        print(f"Assistant: {response}\n")

if __name__ == "__main__":
    run_qa_assistant()
